In [ ]:
import json
from openai import OpenAI

In [ ]:
# Step 1: Initialize the OpenAI client
client = OpenAI()

# Step 2: Define the schema with confidence scores
classification_schema = {
    "type": "object",
    "properties": {
        "themes": {
            "type": "array",
            "description": "List of theme classifications with confidence scores.",
            "items": {
                "type": "object",
                "properties": {
                    "label": {
                        "type": "string",
                        "enum": [
                            "Politics", "Economy", "Environment",
                            "Health", "Technology", "Culture", "Other"
                        ]
                    },
                    "confidence": {
                        "type": "number",
                        "minimum": 0.0,
                        "maximum": 1.0,
                        "description": "Confidence level between 0 and 1"
                    }
                },
                "required": ["label", "confidence"]
            }
        },
        "reasoning": {
            "type": "string",
            "description": "Explanation for why the selected themes were chosen."
        }
    },
    "required": ["themes", "reasoning"],
    "additionalProperties": False
}

# Step 3: Send a request to the Responses API
response = client.responses.create(
    model="gpt-4o-2024-08-06",
    input=[
        {
            "role": "system",
            "content": "You are an assistant that classifies text by theme and gives a confidence score for each classification."
        },
        {
            "role": "user",
            "content": (
                "Classify the following text into one or more of the following themes:\n"
                "Politics, Economy, Environment, Health, Technology, Culture, Other.\n\n"
                "Text: The government passed a new bill to regulate carbon emissions and promote clean energy investments.\n\n"
                "For each relevant theme, include a confidence score (0.0 to 1.0).\n"
                "Also, explain briefly why these themes were selected."
            )
        }
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "theme_classifier_with_confidence",
            "description": "Classifies text into themes with confidence and reasoning",
            "schema": classification_schema,
            "strict": True
        }
    }
)

# Step 4: Parse and print the response
output = json.loads(response.output_text)

print("Themes with Confidence:")
for theme in output["themes"]:
    print(f"- {theme['label']}: {theme['confidence']:.2f}")

print("\nReasoning:")
print(output["reasoning"])